In [85]:
#### Install Libs

!pip install ccxt
!pip install schedule
## TA - LIB ##
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 153, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 328, in run
    session = self.get_default_session(options)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 72, in get_default_session
    self._session = self.enter_context(self._build_session(options))
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 84, in _build_session
    index_urls=self._get_index_urls(options),
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/network/session.py", line 240, in __init__
    self.headers["User-Agent"] = user_agent()
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/network/session.py", line 147, in user_agent
    if platform.system():
  File "/usr/lib/python

In [86]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import seaborn as sns
import math
import ccxt
import matplotlib.pyplot as plt
%matplotlib inline
import talib as ta
import requests
import schedule
import warnings


pd.options.display.max_columns = None
warnings.filterwarnings('ignore')
# sns.set_style('darkgrid')



In [87]:
######### SET UP EXCHANGE ########
apiKey=  ''
secret = ''
sub_account = ''

exchange  = ccxt.ftx({
    'verbose': False,
    'apiKey': apiKey,
    'secret': secret,
    'enableRateLimit': True,
    'headers': {
        'FTX-SUBACCOUNT': sub_account,
    },
})


# mpf.available_styles()
### SETTING ###
symbol= "UNI-PERP"
timeframe ='1d'
###########


In [88]:
def get_data(symbol):
  df= pd.DataFrame(exchange.fetch_ohlcv(symbol,timeframe),columns=['date','open','high','low','close','volume'])
  # df.drop(columns='volume',inplace=True)
  df['volume'] =(df['volume'].astype('int'))
  df['date']=df['date'].apply(exchange.iso8601)
  df['date']=pd.to_datetime(df['date'])
#   df['date']=df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M'))
#   df['date']=pd.to_datetime(df['date'])
  df.set_index('date',inplace=True)

#   df.dropna(inplace=True)
  return df



def anlysis(dataframe):
    try:
        df =dataframe
        up, mid, low = ta.BBANDS(df['close'], timeperiod=50, nbdevup=2, nbdevdn=2, matype=0)
        #   df['chg'] =df.close - df.close.shift(1)
        df['atr'] = ta.ATR(df['high'],df['low'],df['close'],timeperiod= 10)
        df['rsi']= ta.RSI(df['close'],timeperiod=10)
        df.dropna(inplace=True)
#         print(df.tail(5))
        return df
    except Exception as e:
        print(str(e))
        
def sendtext(message):
    try:    
        token = '' ### ใส่ Token ตรงนี้ครับผม
        url = 'https://notify-api.line.me/api/notify'
        header = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer '+token}   
        command = {'message':message}
        return requests.post(url, headers=header, data = command)
    except Exception as e:
        print(e)
        

In [89]:
def AlertPrice(trigger,symbol) :### Trigger x.x % 
    df = get_data(symbol)
    df= anlysis(df)
    now_price = df['close'][-1]
    rsi = df['rsi'][-1]
    atr = df['atr'][-1]
    ref_price = df['close'][-2]
    diff = (now_price - ref_price) /ref_price  *100
    diff = round(diff,3)



    if abs(diff) >= trigger :
        time_stamp = dt.datetime.now().strftime('%D %H:%M:%S')
        msg = f' Alert :{time_stamp} \n•{symbol} \n•Price {ref_price}:>>>{now_price}>>{diff}% \n•RSI : {rsi:.2f} ATR : {(atr/now_price*100):.2f}%'
        print('#'*60,'\n')
        print(msg)
        print('#'*60,'\n')

        sendtext(msg)
        

    else:
        time_stamp = dt.datetime.now().strftime('%D %H:%M:%S')
        msg = f' Check Price :{time_stamp} \n•{symbol} \n•Price {ref_price}:>>>{now_price}>>{diff}% \n•RSI : {rsi:.2f} ATR : {(atr/now_price*100):.2f}%'
        print('#'*60,'\n')
        print(msg)
        print('#'*60,'\n')




In [91]:
    AlertPrice(1.0,symbols)


############################################################ 

 Check Price :03/23/21 17:03:44 
•UNI-PERP 
•Price 34.194:>>>34.143>>-0.149% 
•RSI : 67.39 ATR : 1.10%
############################################################ 



In [ ]:

symbols = 'UNI-PERP'
timeframe = '15m'

while True:
    AlertPrice(1.0,symbols)
    time.sleep(300)

In [ ]:
############ Multi Symbols ##########

symbols = ['UNI-PERP','ETH-PERP','BTC-PERP']
timeframe = '1h'
while True:

  for i in symbols:
    if i == 'UNI-PERP':
      AlertPrice(2.0,i)
    else : 
        AlertPrice(1.5,i)
  time.sleep(300)
